# Multigrid as a Neural Network



In [1]:
%matplotlib inline
import numpy
from matplotlib import pyplot
pyplot.style.use('ggplot')

def Laplacian(n, stencil=[-1, 2, -1], periodic=True):
    A = stencil[1] * numpy.eye(n) + stencil[2] * numpy.eye(n, k=1) + stencil[0] * numpy.eye(n, k=-1)
    if periodic:
        A[0,-1] = stencil[0]
        A[-1,0] = stencil[2]
    return A

Laplacian(4)

array([[ 2., -1.,  0., -1.],
       [-1.,  2., -1.,  0.],
       [ 0., -1.,  2., -1.],
       [-1.,  0., -1.,  2.]])

In [2]:
def Interpolation(A):
    """Interpolation using the exact construction.
    This maintains sparsity in 1D, but the approach will not in higher dimensions.
    """
    Aff = A[1::2, 1::2]
    Afc = A[1::2, 0::2]
    Acf = A[0::2, 1::2]
    nf, nc = Afc.shape
    P = numpy.zeros((nf+nc, nc))
    P[::2] = numpy.eye(nc)
    # Aff is diagonal in our examples so this solve is trivial
    P[1::2] = -numpy.linalg.solve(Aff, Afc)
    R = numpy.zeros((nc, nf+nc))
    R[:,::2] = numpy.eye(nc)
    R[:,1::2] = -numpy.linalg.solve(Aff, Acf.T).T
    return P, R

A = Laplacian(6, periodic=True)
Interpolation(A)

(array([[ 1. ,  0. ,  0. ],
        [ 0.5,  0.5, -0. ],
        [ 0. ,  1. ,  0. ],
        [-0. ,  0.5,  0.5],
        [ 0. ,  0. ,  1. ],
        [ 0.5, -0. ,  0.5]]), array([[ 1. ,  0.5,  0. , -0. ,  0. ,  0.5],
        [ 0. ,  0.5,  1. ,  0.5,  0. , -0. ],
        [ 0. , -0. ,  0. ,  0.5,  1. ,  0.5]]))

In [3]:
def randmean0(shape):
    x = numpy.random.rand(shape)
    return x - numpy.mean(x)

def Additive2Level(A):
    P, R = Interpolation(A)
    Ac = R @ A @ P
    # Use pseudo-inverse because of the null space for periodic.
    # We will replace with a recursive call below.
    Acinv = numpy.linalg.pinv(Ac)
    D = 1/A.diagonal()
    D[::2] = 0
    def apply(b):
        return P @ Acinv @ R @ b + D * b
    return apply

def Test(A, ntests=1, MG=Additive2Level, **kwargs):
    apply = MG(A, **kwargs)
    for i in range(ntests):
        x = randmean0(len(A))
        b = A @ x
        x2 = apply(b)
        # This projection is needed in the periodic case
        x3 = x2 - numpy.mean(x2)
        print('{}: |e2|={:8e} |e3|={:8e}'.format(i,
                                                 numpy.linalg.norm(x2 - x), 
                                                 numpy.linalg.norm(x3 - x)))

A = Laplacian(5, periodic=False)
Test(A, ntests=5)

0: |e2|=2.482534e-16 |e3|=2.203031e-16
1: |e2|=2.063081e-16 |e3|=3.388000e-16
2: |e2|=1.861901e-16 |e3|=1.867065e-16
3: |e2|=2.435542e-16 |e3|=2.579785e-16
4: |e2|=5.003708e-17 |e3|=3.804113e-17


In [4]:
A = Laplacian(5, periodic=True)
Test(A, ntests=5)

0: |e2|=2.718043e-01 |e3|=1.841097e-16
1: |e2|=1.720519e-01 |e3|=1.056901e-16
2: |e2|=2.702989e-01 |e3|=8.777084e-17
3: |e2|=7.737069e-02 |e3|=3.925231e-17
4: |e2|=2.713458e-01 |e3|=7.076311e-17


In [5]:
def AdditiveMG(A, level, verbose=False):
    print('Level {} size {}'.format(level, len(A)))
    if level == 0:
        def apply(b):
            return numpy.linalg.pinv(A) @ b
        return apply
    P, R = Interpolation(A)
    Ac = R @ A @ P
    Acinv = AdditiveMG(Ac, level-1)
    D = 1/A.diagonal()
    D[::2] = 0
    def apply(b):
        y = P @ Acinv(R @ b) + D * b
        return y - numpy.mean(y)
    return apply

A = Laplacian(33, periodic=False)
Test(A, MG=AdditiveMG, level=5, verbose=True)

Level 5 size 33
Level 4 size 17
Level 3 size 9
Level 2 size 5
Level 1 size 3
Level 0 size 2
0: |e2|=1.445690e-15 |e3|=1.445819e-15


In [6]:
A = Laplacian(32, periodic=True)
Test(A, MG=AdditiveMG, level=5, verbose=True)

Level 5 size 32
Level 4 size 16
Level 3 size 8
Level 2 size 4
Level 1 size 2
Level 0 size 1
0: |e2|=1.152953e-15 |e3|=1.157631e-15


In [7]:
# This tests the non-symmetric case, here with
# advection-diffusion discretized using a centered difference.
A = Laplacian(32, stencil=[-2, 2, 0], periodic=True)
Test(A, MG=AdditiveMG, level=5, verbose=True)

Level 5 size 32
Level 4 size 16
Level 3 size 8
Level 2 size 4
Level 1 size 2
Level 0 size 1
0: |e2|=3.671718e-17 |e3|=1.169363e-16


## Towards neural networks

We would like to use data $(x, b=Ax)$ to train an interpolation operator $P$ and restriction $R$ for this multigrid method.